In [ ]:
# # Reinstall the corrupted packages
# !pip install PyYAML attrs soupsieve --force-reinstall
!pip install PyGithub
!pip  install sovai[full]
!pip install papermill
!pip install datawrapper

  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (751 kB)
Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
  Attempting uninstall: soupsieve
    Found existing installation: soupsieve 2.6
    Uninstalling soupsieve-2.6:
      Successfully uninstalled soupsieve-2.6
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2
  Attempting uninstall: attrs
    Found existing installation: attrs 24.2.0
    Uninstalling attrs-24.2.0:
      Successfully uninstalled attrs-24.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [ ]:
# import sovai as sov
# sov.token_auth(token="22fd808d-0947-45f1-995a-82346b921f0f")
# sov.data("short/volume", full_history=True)

In [ ]:
import os
import sys
import argparse
from google.cloud import storage
import tempfile
import subprocess
import papermill as pm
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def parse_arguments():
    """
    Parses command-line arguments.
    """
    parser = argparse.ArgumentParser(description="Execute Jupyter Notebooks from GCS using Papermill.")
    parser.add_argument(
        '--type',
        type=str,
        choices=['monthly', 'weekly'],
        required=True,
        help='Specify the type of notebooks to run: "monthly" or "weekly".'
    )
    return parser.parse_args()

def initialize_gcs_client():
    """
    Initializes the Google Cloud Storage client.
    """
    try:
        client = storage.Client()
        logging.info("Initialized Google Cloud Storage client.")
        return client
    except Exception as e:
        logging.error(f"Error initializing GCS client: {e}")
        sys.exit(1)

import re

def list_notebooks(bucket, folder_prefix):
    """
    Lists all .ipynb files in the specified GCS folder and sorts them numerically.
    """
    try:
        blobs = bucket.list_blobs(prefix=folder_prefix)
        notebooks = [blob.name for blob in blobs if blob.name.endswith('.ipynb')]
        logging.info(f"Found {len(notebooks)} notebooks in folder '{folder_prefix}'.")

        # Define a helper function to extract numbers from the filename
        def extract_number(filename):
            # Adjust the regex pattern based on your filename structure
            match = re.search(r'(\d+)', filename)
            if match:
                return int(match.group(1))
            else:
                # If no number is found, return a large number to push it to the end
                return float('inf')

        # Sort the notebooks based on the extracted number
        notebooks_sorted = sorted(notebooks, key=extract_number)
        logging.info(f"Sorted notebooks: {notebooks_sorted}")
        return notebooks_sorted
    except Exception as e:
        logging.error(f"Error listing notebooks in folder '{folder_prefix}': {e}")
        sys.exit(1)


def download_notebook(bucket, source_blob_name, destination_file_path):
    """
    Downloads a single notebook from GCS to the local filesystem.
    """
    try:
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename(destination_file_path)
        logging.info(f"Downloaded {source_blob_name} to {destination_file_path}")
    except Exception as e:
        logging.error(f"Failed to download {source_blob_name}: {e}")
        raise

def install_packages(packages):
    """
    Installs the required Python packages using pip.
    """
    try:
        logging.info("Upgrading pip and setuptools...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools"])
        logging.info("pip and setuptools upgraded successfully.\n")

        # Force reinstall PyYAML to fix any corruption
        logging.info("Force reinstalling PyYAML to fix potential corruption...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--force-reinstall", "--ignore-installed", "PyYAML"])
        logging.info("PyYAML reinstalled successfully.\n")

        logging.info(f"Installing packages: {', '.join(packages)}")
        subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages)
        logging.info("Package installation successful.")
    except subprocess.CalledProcessError as e:
        logging.error(f"Error installing packages: {e}")
        sys.exit(1)

def execute_notebook_papermill(input_path, output_path, parameters=None):
    """
    Executes a Jupyter notebook using Papermill and saves the output.
    """
    try:
        pm.execute_notebook(
            input_path,
            output_path,
            parameters=parameters or {}
        )
        logging.info(f"Executed and saved notebook to {output_path}")
    except Exception as e:
        logging.error(f"Error executing notebook {input_path}: {e}")
        raise

def upload_notebook(bucket, source_file_path, destination_blob_name):
    """
    Uploads a single notebook to GCS.
    """
    try:
        blob = bucket.blob(destination_blob_name)
        blob.upload_from_filename(source_file_path)
        logging.info(f"Uploaded {source_file_path} to gs://{bucket.name}/{destination_blob_name}")
    except Exception as e:
        logging.error(f"Failed to upload {source_file_path}: {e}")
        raise

# Parse command-line arguments
# If running inside a Jupyter Notebook, you might need to adjust how arguments are parsed.
# For simplicity, we'll set notebook_type directly.
# notebook_type = parse_arguments().type
notebook_type = "weekly"  # Replace with "monthly" as needed

# Configuration
BUCKET_NAME = 'sovai-mockingbird'  # Replace with your GCS bucket name
FOLDER_PREFIX = f"{notebook_type}/"  # 'weekly/' or 'monthly/'

# Initialize GCS client
client = initialize_gcs_client()

# Access the bucket
try:
    bucket = client.get_bucket(BUCKET_NAME)
    logging.info(f"Accessed bucket: {BUCKET_NAME}")
except Exception as e:
    logging.error(f"Error accessing bucket '{BUCKET_NAME}': {e}")
    sys.exit(1)

# List notebooks to run
notebooks = list_notebooks(bucket, FOLDER_PREFIX)

if not notebooks:
    logging.info(f"No notebooks found in folder '{FOLDER_PREFIX}'. Exiting.")
    sys.exit(0)

# Define required packages
required_packages = [
    "sovai[full]",  # Ensure this is the correct package name
    "datawrapper",
    "papermill"
]

# Install required packages
install_packages(required_packages)

# Create a temporary directory to store notebooks
with tempfile.TemporaryDirectory() as temp_dir:
    logging.info(f"Created temporary directory at {temp_dir}")

    for notebook_blob_name in notebooks:
        # Extract the notebook filename
        notebook_filename = os.path.basename(notebook_blob_name)

        # Define local paths
        local_notebook_path = os.path.join(temp_dir, notebook_filename)
        executed_notebook_path = os.path.join(temp_dir, f"executed_{notebook_filename}")

        # Download the notebook
        try:
            download_notebook(bucket, notebook_blob_name, local_notebook_path)
        except Exception:
            logging.warning(f"Skipping execution for {notebook_filename} due to download failure.")
            continue  # Skip to the next notebook

        # Execute the notebook using Papermill
        try:
            execute_notebook_papermill(local_notebook_path, executed_notebook_path)
        except Exception:
            logging.warning(f"Skipping upload for {notebook_filename} due to execution failure.")
            continue  # Skip to the next notebook

        # Define the destination blob name for the executed notebook
        executed_blob_name = os.path.join(FOLDER_PREFIX, f"executed_{notebook_filename}").replace("\\", "/")

        ## I don't for now, want to upload the executed notebook.
        # # Upload the executed notebook back to GCS
        # try:
        #     upload_notebook(bucket, executed_notebook_path, executed_blob_name)
        # except Exception:
        #     logging.warning(f"Failed to upload executed notebook for {notebook_filename}.")
        #     continue  # Continue with the next notebook

        # Optionally, remove the local executed notebook
        try:
            os.remove(executed_notebook_path)
            logging.info(f"Removed local file {executed_notebook_path}")
        except Exception as e:
            logging.error(f"Failed to remove local file {executed_notebook_path}: {e}")

print("Notebook execution process completed.")


  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (751 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.1 which is incompatible.
ydata-profiling 4.8.3 requires numpy<2,>=1.16.0, but you have numpy 2.0.2 which is incompatible.


Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

[IPKernelApp] WARNING | Error in loading extension: beatrix_jupyterlab
Check your config files in /home/jupyter/.ipython/profile_default
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/shellapp.py", line 282, in init_extensions
    self.shell.extension_manager.load_extension(ext)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 62, in load_extension
    return self._load_extension(module_str)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 77, in _load_extension
    mod = import_module(module_str)
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError:

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

[IPKernelApp] WARNING | Error in loading extension: beatrix_jupyterlab
Check your config files in /home/jupyter/.ipython/profile_default
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/shellapp.py", line 282, in init_extensions
    self.shell.extension_manager.load_extension(ext)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 62, in load_extension
    return self._load_extension(module_str)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 77, in _load_extension
    mod = import_module(module_str)
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError:

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

[IPKernelApp] WARNING | Error in loading extension: beatrix_jupyterlab
Check your config files in /home/jupyter/.ipython/profile_default
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/shellapp.py", line 282, in init_extensions
    self.shell.extension_manager.load_extension(ext)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 62, in load_extension
    return self._load_extension(module_str)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 77, in _load_extension
    mod = import_module(module_str)
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError:

KeyboardInterrupt: 

  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (751 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.1 which is incompatible.
ydata-profiling 4.8.3 requires numpy<2,>=1.16.0, but you have numpy 2.0.2 which is incompatible.


Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

[IPKernelApp] WARNING | Error in loading extension: beatrix_jupyterlab
Check your config files in /home/jupyter/.ipython/profile_default
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/shellapp.py", line 282, in init_extensions
    self.shell.extension_manager.load_extension(ext)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 62, in load_extension
    return self._load_extension(module_str)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 77, in _load_extension
    mod = import_module(module_str)
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError:

Executing:   0%|          | 0/7 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/8 [00:00<?, ?cell/s]

[IPKernelApp] WARNING | Error in loading extension: beatrix_jupyterlab
Check your config files in /home/jupyter/.ipython/profile_default
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/shellapp.py", line 282, in init_extensions
    self.shell.extension_manager.load_extension(ext)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 62, in load_extension
    return self._load_extension(module_str)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 77, in _load_extension
    mod = import_module(module_str)
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError:

Executing:   0%|          | 0/14 [00:00<?, ?cell/s]

[IPKernelApp] WARNING | Error in loading extension: beatrix_jupyterlab
Check your config files in /home/jupyter/.ipython/profile_default
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/shellapp.py", line 282, in init_extensions
    self.shell.extension_manager.load_extension(ext)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 62, in load_extension
    return self._load_extension(module_str)
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/extensions.py", line 77, in _load_extension
    mod = import_module(module_str)
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError:

Notebook execution process completed.


In [ ]:
# import nbformat
# from nbconvert.preprocessors import ExecutePreprocessor
# from google.cloud import storage
# import os
# import tempfile

# # Initialize GCS client
# client = storage.Client()

# # Configuration
# BUCKET_NAME = 'sovai-mockingbird'  # Replace with your GCS bucket name
# NOTEBOOKS_FOLDER = 'monthly/'  # Folder in the bucket where notebooks are stored
# NOTEBOOKS_FOLDER = 'weekly/'  # Folder in the bucket where notebooks are stored

# LOCAL_TEMP_DIR = tempfile.gettempdir()


# def download_notebook(bucket_name, source_blob_name, destination_file_name):
#     """Downloads a notebook from GCS to a local temporary directory."""
#     bucket = client.bucket(bucket_name)
#     blob = bucket.blob(source_blob_name)
#     blob.download_to_filename(destination_file_name)
#     print(f"Downloaded {source_blob_name} to {destination_file_name}")

# def run_notebook(notebook_path):
#     """Executes a notebook and saves the output."""
#     with open(notebook_path, "r", encoding="utf-8") as f:
#         notebook = nbformat.read(f, as_version=4)

#     ep = ExecutePreprocessor(timeout=600, kernel_name="python3")

#     try:
#         ep.preprocess(notebook, {"metadata": {"path": os.path.dirname(notebook_path)}})
#         output_path = notebook_path.replace('.ipynb', '_executed.ipynb')
#         with open(output_path, "w", encoding="utf-8") as f:
#             nbformat.write(notebook, f)
#         print(f"Successfully ran and saved: {output_path}")
#     except Exception as e:
#         print(f"Error running notebook {notebook_path}: {e}")

# def upload_notebook(bucket_name, source_file_name, destination_blob_name):
#     """Uploads the executed notebook back to GCS."""
#     bucket = client.bucket(bucket_name)
#     blob = bucket.blob(destination_blob_name)
#     blob.upload_from_filename(source_file_name)
#     print(f"Uploaded {source_file_name} to {destination_blob_name}")

# # Run each notebook in sequence
# for notebook in notebooks_to_run:
#     source_blob = os.path.join(NOTEBOOKS_FOLDER, notebook)
#     local_notebook = os.path.join(LOCAL_TEMP_DIR, notebook)

#     # Download the notebook from GCS
#     download_notebook(BUCKET_NAME, source_blob, local_notebook)

#     # Execute the notebook
#     run_notebook(local_notebook)

#     # Upload the executed notebook back to GCS (optional)
#     executed_notebook = local_notebook.replace('.ipynb', '_executed.ipynb')
#     destination_blob = os.path.join(NOTEBOOKS_FOLDER, executed_notebook.split('/')[-1])
#     upload_notebook(BUCKET_NAME, executed_notebook, destination_blob)

#     # Clean up local files
#     os.remove(local_notebook)
#     os.remove(executed_notebook)
